## Deliverable 2. Create a Customer Travel Destinations Map.

In [11]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [12]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Thinadhoo,MV,0.5333,72.9333,82.56,69,39,9.51,scattered clouds
1,1,Hithadhoo,MV,-0.6000,73.0833,81.23,75,100,9.51,overcast clouds
2,2,Caravelas,BR,-17.7125,-39.2481,73.81,86,100,10.83,overcast clouds
3,3,Zhashkiv,UA,49.2454,30.1102,57.06,53,59,13.47,broken clouds
4,4,Ushuaia,AR,-54.8000,-68.3000,38.86,87,75,27.63,broken clouds


In [13]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Get the maximum temperature.
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [14]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Thinadhoo,MV,0.5333,72.9333,82.56,69,39,9.51,scattered clouds
1,1,Hithadhoo,MV,-0.6000,73.0833,81.23,75,100,9.51,overcast clouds
8,8,Butaritari,KI,3.0707,172.7902,82.96,72,27,13.96,scattered clouds
13,13,Buchanan,LR,5.8808,-10.0467,76.96,88,100,4.25,moderate rain
25,25,San Onofre,CO,9.7359,-75.5263,78.53,86,55,2.35,broken clouds
...,...,...,...,...,...,...,...,...,...,...
669,669,Tete,MZ,-16.1564,33.5867,77.09,50,0,4.61,clear sky
671,671,Kidal,ML,18.4411,1.4078,83.28,20,2,9.04,clear sky
674,674,Teknaf,BD,20.8624,92.3058,76.17,90,93,3.44,overcast clouds
678,678,Biak,ID,-0.9131,122.8766,79.57,75,100,3.94,overcast clouds


In [15]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                201
City                   201
Country                200
Lat                    201
Lng                    201
Max Temp               201
Humidity               201
Cloudiness             201
Wind Speed             201
Weather Description    201
dtype: int64

In [16]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_clean_df = preferred_cities_df.dropna()

In [17]:
preferred_cities_clean_df.shape

(200, 10)

In [18]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_clean_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,Thinadhoo,MV,82.56,scattered clouds,0.5333,72.9333,
1,Hithadhoo,MV,81.23,overcast clouds,-0.6000,73.0833,
8,Butaritari,KI,82.96,scattered clouds,3.0707,172.7902,
13,Buchanan,LR,76.96,moderate rain,5.8808,-10.0467,
25,San Onofre,CO,78.53,broken clouds,9.7359,-75.5263,
29,Cam Ranh,VN,75.16,scattered clouds,11.9214,109.1591,
30,Vaini,TO,78.96,scattered clouds,-21.2000,-175.2000,
42,Taoudenni,ML,76.82,clear sky,22.6783,-3.9836,
46,Mangrol,IN,76.37,clear sky,21.1167,70.1167,
47,San Patricio,US,84.92,scattered clouds,28.0170,-97.5169,


In [19]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels['results'][0]['name']
#         print(f"hotel : {hotels['results'][0]['name']}")
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [20]:
hotels

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': -8.459281799999998, 'lng': -76.4624779},
    'viewport': {'northeast': {'lat': -8.457983419708494,
      'lng': -76.46111956970849},
     'southwest': {'lat': -8.460681380291499, 'lng': -76.4638175302915}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'icon_background_color': '#909CE1',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
   'name': 'Hotel San Pedro',
   'opening_hours': {'open_now': True},
   'photos': [{'height': 2268,
     'html_attributions': ['<a href="https://maps.google.com/maps/contrib/104861132896879107491">Hotel San Pedro</a>'],
     'photo_reference': 'Aap_uEBsZDAMtJZgAgzs9XfFeh-aEXaevYtcoxuxcjUiq8t34c0aTEdoyGpTeX9g4CwUoNziUDtnc9pRTwZni-4s0yt4hu4Fytsr8guIl30mY2j_m2TyYf2s9NfJRQCGAMTMJZwg7KbvQQgFtEDK-s0Ldi3_egvY3Rccu3wv-Y-dXDcIb5_9',
     'width': 4032}],
   'place

In [21]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City                   200
Country                200
Max Temp               200
Weather Description    200
Lat                    200
Lng                    200
Hotel Name             200
dtype: int64

In [22]:
import numpy as np

In [23]:
hotel_df.replace("", np.nan, inplace = True)

In [24]:
hotel_clean_df = hotel_df.dropna()

In [25]:
hotel_clean_df

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
1,Hithadhoo,MV,81.23,overcast clouds,-0.6000,73.0833,Scoop Guest House
8,Butaritari,KI,82.96,scattered clouds,3.0707,172.7902,Isles Sunset Lodge
13,Buchanan,LR,76.96,moderate rain,5.8808,-10.0467,Kamaneahn Hotel Inc
25,San Onofre,CO,78.53,broken clouds,9.7359,-75.5263,Hotel Sinai
29,Cam Ranh,VN,75.16,scattered clouds,11.9214,109.1591,OYO 598 Peony Hotel
...,...,...,...,...,...,...,...
667,Manaure,CO,85.05,few clouds,11.7751,-72.4445,Hotel Palaaima
669,Tete,MZ,77.09,clear sky,-16.1564,33.5867,Zambeze Paraíso Misterioso
674,Teknaf,BD,76.17,overcast clouds,20.8624,92.3058,Forestry Guesthouse
678,Biak,ID,79.57,overcast clouds,-0.9131,122.8766,Penginapan Kim


In [26]:
hotel_clean_df.count()

City                   185
Country                185
Max Temp               185
Weather Description    185
Lat                    185
Lng                    185
Hotel Name             185
dtype: int64

In [27]:
# 8a. Create the output File (CSV)

output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_clean_df.to_csv(output_data_file, index_label="City_ID")

In [28]:
#Configure gmaps to use your Googe API key 
gmaps.configure(api_key=g_key)

In [29]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Weather Description</dt><dd>{Weather Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_clean_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_clean_df[["Lat", "Lng"]]

In [30]:
# 11a. Add a marker layer for each city to the map. 

#add a pop-up marker for each city.
locations = hotel_clean_df[["Lat", "Lng"]]
max_temp = hotel_clean_df["Max Temp"]
city_weather_descrip = hotel_clean_df["Weather Description"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))